# 第2回 金融データ活用チャレンジ ベースライン notebook

第2回 金融データ活用チャレンジ のベースラインを作成してみます。

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com//nishimoto/SignateLoan2nd/blob/main/Signate_%E7%AC%AC2%E5%9B%9E_%E9%87%91%E8%9E%8D%E3%83%86%E3%82%99%E3%83%BC%E3%82%BF%E6%B4%BB%E7%94%A8%E3%83%81%E3%83%A3%E3%83%AC%E3%83%B3%E3%82%B7%E3%82%99_%E3%83%98%E3%82%99%E3%83%BC%E3%82%B9%E3%83%A9%E3%82%A4%E3%83%B3.ipynb
)

コンペURL: https://signate.jp/competitions/1325

コンペ課題概要：企業向けローンの返済可否予測

Public score: 0.6738

**このnotebookを動かすために必要なこと**：2個目のセルにある `path_in_folder` のフォルダ内に`train.csv`, `test.csv`, `sample_submission.csv`を置いてください。デフォルトだとカレントフォルダになっているので、colabからデータアップロードすればOKかと


In [1]:
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import lightgbm as lgb
from sklearn import metrics
from sklearn.metrics import f1_score, roc_auc_score
from sklearn.model_selection import StratifiedKFold

# warningsを非表示にする
warnings.filterwarnings("ignore")

In [2]:
# Config cell
target = "MIS_Status"
path_in_folder = "./"

cols_category = [
    "RevLineCr",
    "LowDoc",
    "Sector",
    "State",
    "BankState",
    "FranchiseCode",
]

In [3]:
df_train = pd.read_csv(f"{path_in_folder}/train.csv", index_col=0)
df_test = pd.read_csv(f"{path_in_folder}/test.csv", index_col=0)
ss = pd.read_csv(f"{path_in_folder}/sample_submission.csv", header=None)

In [4]:
# コンペ終了後データの中身見れない状態の方がよさそうなのでコメントアウト。みたい場合は以下コメントアウト外してください
# df_train

---

## EDA

まずは各列の基礎的な統計情報（列の型、NaNがある行数、値の種類）を見てみます。

In [5]:
rows = []
for col in df_train.columns:
    rows.append([col, df_train[col].dtype, df_train[col].isnull().sum(), len(df_train[col].unique())])
pd.DataFrame(rows, columns=["列名", "列の型", "NaNである行の数", "値の種類"])

,列名,列の型,NaNである行の数,値の種類
0,Term,int64,0,228
1,NoEmp,int64,0,196
2,NewExist,float64,0,2
3,CreateJob,int64,0,49
4,RetainedJob,int64,0,83
5,FranchiseCode,int64,0,271
6,RevLineCr,object,1079,5
7,LowDoc,object,531,7
8,DisbursementDate,object,150,917
9,MIS_Status,int64,0,2


In [6]:
# 値の種類が多くないObject型の場合はどんな種類があるかも見てみます
for col in ["RevLineCr", "LowDoc", "State", "BankState"]:
    print(col, df_train[col].unique())

RevLineCr ['N' '0' 'Y' nan 'T']
LowDoc ['N' 'C' nan 'Y' '0' 'S' 'A']
State ['AZ' 'OK' 'NJ' 'TN' 'CA' 'IA' 'TX' 'NH' 'ND' 'MD' 'PA' 'UT' 'WI' 'WY'
 'OH' 'NY' 'SD' 'RI' 'GA' 'DC' 'NV' 'FL' 'IN' 'VA' 'ME' 'CT' 'VT' 'MT'
 'MO' 'AL' 'MN' 'KY' 'NE' 'OR' 'IL' 'KS' 'MA' 'CO' 'MI' 'NC' 'WA' 'LA'
 'ID' 'WV' 'NM' 'AR' 'AK' 'MS' 'DE' 'HI' 'SC']
BankState ['SD' 'OK' 'NJ' 'CA' 'IA' 'NH' 'ND' 'AZ' 'RI' 'UT' 'WI' 'DE' 'WY' 'NC'
 'OH' 'IL' 'MD' 'CT' 'TN' 'PA' 'IN' 'VA' 'ME' 'NY' 'AL' 'VT' 'MT' 'MO'
 'MN' 'NE' 'DC' 'TX' 'MI' 'KY' 'CO' 'OR' 'KS' 'FL' 'WA' 'LA' 'MA' 'GA'
 'ID' 'NV' 'AR' 'AK' 'WV' 'MS' 'SC' 'HI' 'NM' nan]


---

## 前処理

### カテゴリカル変数の変換

勾配ブースティングでは数字型はそのまま取り扱うことができますが、カテゴリカルな変数は何らか数値に変換して扱う必要があります。

ここでは上記表と見比べながら、変換が必要そうな列を変換する方針を決めていきます。まず、カテゴリカルな変数の説明を大まかな分類に分けた上で書き出してみます。

---

 - 貸借手の所在地系の変数
     - **City**: 借り手の会社の所在地（市）
     - **State**: 借り手の会社の所在地（州）
     - **BankState**: 貸し手の所在地（州）


 - 借り手の会社に関する変数（数値として読み込まれているが、本来カテゴリカルな数字）
     - **Sector**: 産業分類コード
     - **FranchiseCode**: どのブランドのフランチャイズであるかを識別する一意の5桁のコード


 - 今回の借り入れに関する変数
     - **RevLineCr**: リボルビング信用枠か
     - **LowDoc**: 15 万ドル未満のローンを1ページの短い申請で処理できるプログラムか


 - 日付系の変数
     - **DisbursementDate**: 銀行によって支払われた日
     - **ApprovalDate**: 米国中小企業庁の承認日


 - 金額系の変数
     - **DisbursementGross**: 銀行によって支払われた金額
     - **GrAppv**: 銀行によって承認されたローンの総額
     - **SBA_Appv**: SBAが保証する承認されたローンの金額

---

今回のチュートリアルでは、以下のように変換してみたいと思います。

 - 全体方針

... **カテゴリカル変数は基本的にはLabelEncodingとCountEncodingを行う**


 - 貸借手の所在地系の変数

... **City**: Cityは汎用性が低いと考えられるためDrop


 - 借り手の会社に関する変数（Sector, FranchiseCode）

... **Sector**: 公式ページに、31~33は製造業等、同じ意味の数字がいくつかあるため、一部数字は変換を行う

... **Sector**と**FranchiseCode**: カテゴリカル変数へ変換

 - 今回の借り入れに関する変数（RevLineCr, LowDoc）

... 公式ページには値の候補が2つ（YesとNoのYN）と記載があるが、実際の値の種類は2より多い。YN以外はNaNへ置換

 - 日付系の変数（DisbursementDate, ApprovalDate）

... 日付型へ変更 → 年を抽出（借りた月や日にはあまり意味はないと思われるため）

 - 金額系の変数（DisbursementGross, GrAppv, SBA_Appv）

... 数値型へ変更

### その他特徴量エンジニアリング

適当に思いついたものを記載しています。背景にありそうな仮定もカッコで記載しています。

 - 金額の割合を見てみる
     - （SBAが保証する金額に対して借りる金額が小さければリスクは低そう）


 - 借り手と貸し手が同じ州か見てみる
     - （違う州まで借りに行ってるのは財政が厳しい可能性？）


 - SBAの承認年と借りた年の差を見てみる
     - （ここの承認年が長い企業は設立年数が長く、リスクが低いかも）


 - NaNの列数のカウント
     - 一般的にNaNが多いとデフォルトリスクが高いことが多い


In [7]:
# 貸借手の所在地系の変数
# City: Cityは汎用性が低いと考えられるためDrop
df_train.drop("City", axis=1, inplace=True)

In [8]:
# 借り手の会社に関する変数（Sector, FranchiseCode）
# 31-33, 44-45, 48-49 は同じらしい => 32,33を31に, 45を44に, 49を48に変換
code_dict = {
    32: 31,
    33: 31,
    45: 44,
    49: 48
}
df_train["Sector"] = df_train["Sector"].replace(code_dict)

In [9]:
# 今回の借り入れに関する変数（RevLineCr, LowDoc）
# 公式ページには値の候補が2つ（YesとNoのYN）と記載があるが、実際の値の種類は2より多い。YN以外はNaNへ置換
revline_dict = {'0': np.nan, 'T': np.nan}
df_train["RevLineCr"] = df_train["RevLineCr"].replace(revline_dict)

lowdoc_dict = {'C': np.nan, '0': np.nan, 'S': np.nan, 'A': np.nan}
df_train["LowDoc"] = df_train["LowDoc"].replace(lowdoc_dict)

In [10]:
# 日付系の変数（DisbursementDate, ApprovalDate）
# 日付型へ変更 → 年を抽出（借りた月や日にはあまり意味はないと思われるため）
df_train['DisbursementDate'] = pd.to_datetime(df_train['DisbursementDate'], format='%d-%b-%y')
df_train["DisbursementYear"] = df_train["DisbursementDate"].dt.year
df_train.drop(["DisbursementDate", "ApprovalDate"], axis=1, inplace=True)

In [11]:
# 本来数値型のものを変換する
cols = ["DisbursementGross", "GrAppv", "SBA_Appv"]
df_train[cols] = df_train[cols].applymap(lambda x: x.strip().replace('$', '').replace(',', '')).astype(float).astype(int)

In [12]:
# 特徴量エンジニアリング
df_train["FY_Diff"] = df_train["ApprovalFY"] - df_train["DisbursementYear"]
df_train["State_is_BankState"] = (df_train["State"] == df_train["BankState"])
df_train["State_is_BankState"] = df_train["State_is_BankState"].replace({True: 1, False: 0})

df_train['SBA_Portion'] = df_train['SBA_Appv'] / df_train['GrAppv']
df_train["DisbursementGrossRatio"] = df_train["DisbursementGross"] / df_train["GrAppv"]
df_train["MonthlyRepayment"] = df_train["GrAppv"] / df_train["Term"]
df_train["NullCount"] = df_train.isnull().sum(axis=1)

In [13]:
# カテゴリカル変数の設定
# nanと新規値は-1として設定
df_train[cols_category] = df_train[cols_category].fillna(-1)

# countencode, labelencode
# ce_dict: 列名を入れるとそのカテゴリのデータがどのくらいあるかを返してくれます
# replace_dict: 列名を入れるとlabelencodeのための数字を返してくれます
ce_dict = {}
replace_dict = {}
for col in cols_category:
    replace_dict[col] = {}
    vc = df_train[col].value_counts()
    ce_dict[col] = vc
    replace_dict_in_dict = {}
    for i, k in enumerate(vc.keys()):
        replace_dict_in_dict[k] = i
    replace_dict[col] = replace_dict_in_dict
    df_train[f"{col}_CountEncode"] = df_train[col].replace(vc).astype(int)
    df_train[col] = df_train[col].replace(replace_dict_in_dict).astype(int)


### 前処理 - 後確認

正常に変換されているか、簡単な確認をしてみます。

In [14]:
# コンペ終了後データの中身見れない状態の方がよさそうなのでコメントアウト。みたい場合は以下コメントアウト外してください
# df_train

In [15]:
rows = []
for col in df_train.columns:
    rows.append([col, df_train[col].dtype, df_train[col].isnull().sum(), len(df_train[col].unique())])
pd.DataFrame(rows, columns=["列名", "列の型", "NaNである行の数", "値の種類"])

,列名,列の型,NaNである行の数,値の種類
0,Term,int64,0,228
1,NoEmp,int64,0,196
2,NewExist,float64,0,2
3,CreateJob,int64,0,49
4,RetainedJob,int64,0,83
5,FranchiseCode,int64,0,271
6,RevLineCr,int64,0,3
7,LowDoc,int64,0,3
8,MIS_Status,int64,0,2
9,Sector,int64,0,20


### 予測用データの変換

testデータも同様に変換します。今後の改変しやすさを考え、関数化してから変換してみます。

In [16]:
def preprocess(df, replace_dict=None, ce_dict=None):
    # 貸借手の所在地系の変数
    # City: Cityは汎用性が低いと考えられるためDrop
    df.drop("City", axis=1, inplace=True)

    # 借り手の会社に関する変数（Sector, FranchiseCode）
    # 31-33, 44-45, 48-49 は同じらしい => 32,33を31に, 45を44に, 49を48に変換
    code_dict = {
        32: 31,
        33: 31,
        45: 44,
        49: 48
    }
    df["Sector"] = df["Sector"].replace(code_dict)

    # 今回の借り入れに関する変数（RevLineCr, LowDoc）
    # 公式ページには値の候補が2つ（YesとNoのYN）と記載があるが、実際の値の種類は2より多い。YN以外はNaNへ置換
    revline_dict = {'0': np.nan, 'T': np.nan}
    df["RevLineCr"] = df["RevLineCr"].replace(revline_dict)

    lowdoc_dict = {'C': np.nan, '0': np.nan, 'S': np.nan, 'A': np.nan}
    df["LowDoc"] = df["LowDoc"].replace(lowdoc_dict)

    # 日付系の変数（DisbursementDate, ApprovalDate）
    # 日付型へ変更 → 年を抽出（借りた月や日にはあまり意味はないと思われるため）
    df['DisbursementDate'] = pd.to_datetime(df['DisbursementDate'], format='%d-%b-%y')
    df["DisbursementYear"] = df["DisbursementDate"].dt.year
    df.drop(["DisbursementDate", "ApprovalDate"], axis=1, inplace=True)

    # 本来数値型のものを変換する
    cols = ["DisbursementGross", "GrAppv", "SBA_Appv"]
    df[cols] = df[cols].applymap(lambda x: x.strip().replace('$', '').replace(',', '')).astype(float).astype(int)

    # 特徴量エンジニアリング
    df["FY_Diff"] = df["ApprovalFY"] - df["DisbursementYear"]
    df["State_is_BankState"] = (df["State"] == df["BankState"])
    df["State_is_BankState"] = df["State_is_BankState"].replace({True: 1, False: 0})

    df['SBA_Portion'] = df['SBA_Appv'] / df['GrAppv']
    df["DisbursementGrossRatio"] = df["DisbursementGross"] / df["GrAppv"]
    df["MonthlyRepayment"] = df["GrAppv"] / df["Term"]
    df["NullCount"] = df.isnull().sum(axis=1)

    # カテゴリカル変数の設定
    df[cols_category] = df[cols_category].fillna(-1)

    # train
    if replace_dict is None:
        # countencode, labelencode
        # ce_dict: 列名を入れるとそのカテゴリのデータがどのくらいあるかを返してくれます
        # replace_dict: 列名を入れるとlabelencodeのための数字を返してくれます
        ce_dict = {}
        replace_dict = {}
        for col in cols_category:
            replace_dict[col] = {}
            vc = df[col].value_counts()
            ce_dict[col] = vc
            replace_dict_in_dict = {}
            for i, k in enumerate(vc.keys()):
                replace_dict_in_dict[k] = i
            replace_dict[col] = replace_dict_in_dict
            df[f"{col}_CountEncode"] = df[col].replace(vc).astype(int)
            df[col] = df[col].replace(replace_dict_in_dict).astype(int)
        return df, replace_dict, ce_dict

    # test
    else:
        for col in cols_category:
            # カウントエンコード
            test_vals_uniq = df[col].unique()
            ce_dict_in_dict = ce_dict[col]
            for test_val in test_vals_uniq:
                if test_val not in ce_dict_in_dict.keys():
                    ce_dict_in_dict[test_val] = -1
            df[f"{col}_CountEncode"] = df[col].replace(ce_dict_in_dict).astype(int)

            # LabelEncode
            test_vals_uniq = df[col].unique()
            replace_dict_in_dict = replace_dict[col]
            for test_val in test_vals_uniq:
                if test_val not in replace_dict_in_dict.keys():
                    replace_dict_in_dict[test_val] = -1
            df[col] = df[col].replace(replace_dict_in_dict).astype(int)
        return df

In [17]:
# ↑のnotebookの処理はコレでも動く。今回は実行済みのためコメントアウト
# df_train, replace_dict, ce_dict = preprocess(df_train)
df_test = preprocess(df_test, replace_dict=replace_dict, ce_dict=ce_dict)

In [19]:
# コンペ終了後データの中身見れない状態の方がよさそうなのでコメントアウト。みたい場合は以下コメントアウト外してください
# df_test

### 前処理 - 後確認 - 相関係数の確認

特徴量エンジニアリングの簡易的な成果確認のために、target（`MIS_Status`）との相関を見てみます。

NullCountやRevLineCrのCountEncodeが結構働いてそうです。



In [21]:
s_per = df_train.corr("pearson")[target].sort_values()
s_spr = df_train.corr("spearman")[target].sort_values()
df_corr = pd.concat([s_per, s_spr], axis=1)
df_corr.columns = ["Pearson", "Spearman"]

# 平均値でソート
df_corr.loc[df_corr.mean(axis=1).sort_values(ascending=False).keys(), :].drop(target)

,Pearson,Spearman
RevLineCr_CountEncode,0.138850,0.127572
NoEmp,0.092940,0.171855
Term,0.122125,0.119292
LowDoc_CountEncode,0.107665,0.114348
Sector_CountEncode,0.104510,0.105012
DisbursementGrossRatio,0.047301,0.032333
FY_Diff,0.034412,0.042713
State,0.024825,0.026830
BankState,0.009405,0.005933
DisbursementGross,0.000481,0.000447


In [22]:
train_y = df_train[target]
train_x = df_train.drop(target, axis=1)

---

## 学習・評価・予測

勾配ブースティング（LightGBM）による学習を行います。LightGBMは学習時に[`categorical_feature`](https://lightgbm.readthedocs.io/en/latest/Parameters.html#categorical_feature)というパラメーターを指定することでカテゴリカル変数を扱うことができます。

 - LightGBMのハイパラは以下（[自分の書いた記事を自分で参考にした](https://zenn.dev/nishimoto/articles/815e841b188b87)）

```python
params_lgb = {
    "n_estimators": 3000,
    "learning_rate": 0.01,
    "colsample_bytree": 0.8,
    "subsample_freq": 1,
    "subsample": 0.8,
    "random_seed": 0,
}
```

 - CV: Stratified K-Fold（01の割合が同じになるように分割）

 - 01のcutoff: 全通り見てみて、最もf1スコアが高くなるものを算出


In [23]:
params_lgb = {
    "n_estimators": 3000,
    "learning_rate": 0.01,
    "colsample_bytree": 0.8,
    "subsample_freq": 1,
    "subsample": 0.8,
    "random_seed": 0,
}

In [24]:
# f1スコアが最も高くなる点を見つける
from sklearn import metrics
def decide_cutoff(val_y, preds_y_proba):
    mean_f1_list = []
    fpr, tpr, thresholds = metrics.roc_curve(val_y, preds_y_proba)
    for threshold in thresholds:
        preds_y = [1 if prob > threshold else 0 for prob in preds_y_proba]
        mean_f1_list.append(f1_score(val_y, preds_y, average='macro'))
    return np.max(mean_f1_list), thresholds[np.argmax(mean_f1_list)]

In [25]:
list_metrics_auc = []
list_metrics_f1 = []
list_cutoff = []
list_models = []
cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=0)
for fold, (trn_idx, val_idx) in enumerate(cv.split(train_x, train_y), start=1):
    trn_x = train_x.iloc[trn_idx, :]
    trn_y = train_y[trn_idx]
    val_x = train_x.iloc[val_idx, :]
    val_y = train_y[val_idx]
    model_lgb = lgb.LGBMClassifier(**params_lgb)
    model_lgb.fit(
        trn_x, trn_y,
        eval_set=(val_x, val_y),
        callbacks=[lgb.early_stopping(100, verbose=True)],
        categorical_feature=cols_category,
    )
    list_models.append(model_lgb)
    preds_y_proba = model_lgb.predict_proba(val_x)[:, 1]
    auc = roc_auc_score(val_y, preds_y_proba)
    f1, threshold = decide_cutoff(val_y, preds_y_proba)
    list_metrics_auc.append(auc)
    list_metrics_f1.append(f1)
    list_cutoff.append(threshold)
    print(f"Fold: {fold}, AUC: {auc}, f1 score: {f1} Threshold: {threshold}")

[LightGBM] [Info] Number of positive: 25178, number of negative: 3026
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.005978 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2476
[LightGBM] [Info] Number of data points in the train set: 28204, number of used features: 29
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.892710 -> initscore=2.118729
[LightGBM] [Info] Start training from score 2.118729
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[507]	valid_0's binary_logloss: 0.286519
Fold: 1, AUC: 0.7644520551323192, f1 score: 0.6675970219878118 Threshold: 0.7538001356190227
[LightGBM] [Info] Number of positive: 25178, number of negative: 3027
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.005443 seconds.
You can set `force_row_wise=true` to rem

In [26]:
# AUC: 0.77とまあ一定程度予測できている印象
print(np.mean(list_metrics_auc), np.mean(list_metrics_f1), np.median(list_cutoff))

0.7735209741638948 0.6745620947119212 0.7237334150076405


### 予測

In [27]:
threshold = np.median(list_cutoff)
preds_y_proba = np.zeros(len(df_test))
for model in list_models:
    preds_y_proba += model.predict_proba(df_test[model.feature_name_])[:, 1] / len(list_models)
preds_y = [1 if prob > threshold else 0 for prob in preds_y_proba]

In [28]:
ss[1] = preds_y
ss[1] = ss[1].astype(int)
ss.to_csv("submission.csv", header=False, index=False)

In [29]:
ss[1].value_counts()

1    39433
0     2875
Name: 1, dtype: int64

## 終了

`submission.csv` ができているはずなので、これをコンペサイトへ提出することで精度を見ることができます。
